In [1]:
# Libraries
import matplotlib.animation as animation
%matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import os
import pandas as pd
import csv
import numpy as np 

# Change Working Directory
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Blog stuff\\Tide Animation\\data')

# Import and read 
t2015 = pd.read_csv('2015_LJ_LST_v2.csv') 
moon = pd.read_csv('moon_data.csv') 

# Joining the Date and time columns into one datetime vector
t2015['Datetime'] = pd.to_datetime(t2015[['Date','Time (LST/LDT)']]
                   .astype(str).apply(' '.join, 1), format= '%Y/%m/%d %H:%M')
t2015['DateDMY'] = pd.to_datetime(t2015['Date'], format = '%Y/%m/%d') #For just the date

# Make the Datetime the index for the dataframe
t2015.set_index('Datetime')

# Pull out just the hours
t2015['month'] = t2015['Datetime'].dt.month  
t2015['hour'] = t2015['Datetime'].dt.hour

# changed the hours from 0-23 to 1-24
hournew = []
for i in range(0, len(t2015['hour'])):
    hournew.append(t2015['hour'][i] + 1)
t2015['hournew'] = hournew

# replace stuff to get the string ready for a float change
t2015['Verified (ft)'] = t2015['Verified (ft)'].replace(' ', '.')
t2015['Verified (ft)'] = t2015['Verified (ft)'].replace('-',np.nan)
#Change to float
t2015['Verified (ft)'] = t2015['Verified (ft)'].astype(float)

#Turn moon date string into datetime
moon['Date'] = pd.to_datetime(moon['Actual Date'], format = '%m/%d/%Y')

# Print stuff
print(t2015.head())
print('t2015 Date format: ', t2015['DateDMY'].dtypes)
print(moon.head())
print('moon Date format: ', moon['Date'].dtypes)
# print('length of moon values: ', len(t2015['Moon Values']))

Using matplotlib backend: Qt5Agg
         Date Time (LST/LDT) Predicted (ft) Preliminary (ft)  Verified (ft)  \
0  2015/01/01          00:00          1.638                -           1.94   
1  2015/01/01          01:00          1.999                -           2.29   
2  2015/01/01          02:00          2.747                -           3.03   
3  2015/01/01          03:00          3.735                -           4.01   
4  2015/01/01          04:00           4.74                -           5.00   

             Datetime    DateDMY  month  hour  hournew  
0 2015-01-01 00:00:00 2015-01-01      1     0        1  
1 2015-01-01 01:00:00 2015-01-01      1     1        2  
2 2015-01-01 02:00:00 2015-01-01      1     2        3  
3 2015-01-01 03:00:00 2015-01-01      1     3        4  
4 2015-01-01 04:00:00 2015-01-01      1     4        5  
t2015 Date format:  datetime64[ns]
       Moon Type Excel Date Month  Year Actual Date       Date
0      Full Moon   1/4/2020   Sun  2015    1/4/2015 

In [2]:
#Pulls out the full moons and the new moons and puts them into a dataframe
fullmoon = pd.DataFrame()
newmoon = pd.DataFrame()
fullmoondate = []
newmoondate = []
for i in range(0, len(moon['Date'])):
    if moon['Moon Type'][i] == 'Full Moon':
        fullmoondate.append(moon['Date'][i])
    elif moon['Moon Type'][i] == 'New Moon':
        newmoondate.append(moon['Date'][i])

#Add Values/dates to the dataframes
fullmoon = pd.DataFrame({'Date': fullmoondate, 'Values': 100})
newmoon = pd.DataFrame({'Date': newmoondate, 'Values': 0})


In [3]:
daysin2015 = pd.date_range('2015-01-01', periods=365, freq='d')
moonanimate = pd.DataFrame({'Date': daysin2015, 'val': np.nan}) 

#Indicies in tidedata where the full moon data matches it
fullmoonin2015idx = []
newmoonin2015idx = []
for i in range(0, len(moonanimate['Date'])):
    date = moonanimate['Date'][i]
    for j in range(0, len(fullmoon['Date'])):
        datefullmoon = fullmoon['Date'][j]
        if date == datefullmoon:
            fullmoonin2015idx.append(i)
    for k in range(0, len(newmoon['Date'])):
        datenewmoon = newmoon['Date'][k]
        if date == datenewmoon:
            newmoonin2015idx.append(i)
# print(fullmoonin2015idx)
# print(newmoonin2015idx)

# Replace values of moonanimate dataframe with fullmoon or newmoon values based on idx
moonanimate.loc[fullmoonin2015idx, 'val'] = 100
moonanimate.loc[newmoonin2015idx, 'val'] = 0

#Linear interpolation between the values
moonanimatefull = moonanimate.dropna(axis=1, how='all').interpolate()

#fill in the ends (extrapolate)
moonanimatefull.loc[0:2, 'val'] = np.array(moonanimatefull['val'][30:33])
moonanimatefull.loc[359:365, 'val'] = np.array(moonanimatefull['val'][34:40])

print(moonanimatefull['val'])

0       78.571429
1       85.714286
2       92.857143
3      100.000000
4       93.750000
          ...    
360     86.666667
361     80.000000
362     73.333333
363     66.666667
364     60.000000
Name: val, Length: 365, dtype: float64


In [4]:
##MATPLOTLIB ANIMATION FOR THE MOON
%matplotlib notebook
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

#data
x = moonanimatefull['Date']
y = moonanimatefull['val']
n = len(moonanimatefull['Date'])
phase = ('', 'New Moon', '', '', '', 'Full Moon')

#Animation function
def animation_frame(i):
    plt.gca().cla()
    ax.set_title("Moon Phase as bar height", fontsize=22)
    ax.set_ylabel("Moon Phase", fontsize=16)
    ax.set_yticklabels(phase)
    labels = ['']
    heights = y
    plt.yticks(fontsize=14, rotation=90)
    plt.bar(x=labels, height=heights[i], color='#9fbac9', width = 1)
    plt.suptitle('Moon phases as bar height', fontsize=18)
    plt.title(f' Date: {x[i]}') 
    plt.ylim([0, 100])    

#Figure
fig, ax = plt.subplots()
n=len(y) #number of frames

#Calling the Animation function
animation = FuncAnimation(fig, animation_frame, frames=n, interval=2000)

#Saving the plot
os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Blog stuff\\Tide Animation\\gif')
animation.save('moonphases3.gif', fps=10, writer='imagemagick')

MovieWriter imagemagick unavailable; trying to use pillow instead.


TypeError: 'str' object is not callable

In [19]:
##MATPLOTLIB ANIMATION FOR THE TIDE
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
ln, = ax.plot([], [], '#0c8bc2',linewidth=3)

#data
x = np.array(t2015['hournew'])
y = t2015['Verified (ft)']
number_frames = len(np.unique(t2015['Date']))

def init():
    ax.set_xlim(0, 24)
    ax.set_ylim(-2, 8) 
    ax.set_xlabel("Hour of the Day", fontsize=16)
    ax.set_ylabel("Tide Height (ft)", fontsize=16)
    return ln,

def update(i):
    plt.suptitle('Tidal Data (La Jolla, CA)', fontsize=18) 
    ax.set_title('Date: ' + t2015['Date'][i*24])
    ln.set_data(x[i*24: i*24+24], y[i*24: i*24+24])
    return ln,

ani = FuncAnimation(fig, update, frames=number_frames, init_func=init, blit=False, interval=2000)
plt.show()

os.chdir('C:\\Users\\Garrett\\Google Drive\\Documents\\Blog stuff\\Tide Animation\\gif')
ani.save('tide2015gifv3.gif', fps=10, writer='imagemagick')

In [6]:
#Example Animation
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

x = np.arange(0, 2*np.pi, 0.01)        # x-array
line, = ax.plot(x, np.sin(x))

def animate(i):
    line.set_ydata(np.sin(x+i/10.0))  # update the data
    return line,


ani = animation.FuncAnimation(fig, animate, np.arange(1, 200), interval=25, blit=True)
plt.show()